In [1]:
import os

In [2]:
%pwd

'c:\\Users\\layeg\\Desktop\\GitHub\\Holland_Barret\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\layeg\\Desktop\\GitHub\\Holland_Barret'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    """
    Configuration class for model training.

    Attributes:
    root_dir (Path): The root directory where model artifacts will be stored.
    train_data_path (Path): The path to the training data.
    test_data_path (Path): The path to the testing data.
    model_name (str): The name of the model to be trained.
    n_estimators (int): The number of trees in the forest.
    max_depth (int): The maximum depth of the tree.
    learning_rate (int): The learning rate of the model.
    random_state (int): The random state for reproducibility.
    min_samples_split (int): The minimum number of samples required to split an internal node.
    subsample (int): The fraction of samples to be used for fitting the individual base learners.
    min_samples_leaf (int): The minimum number of samples required to be at a leaf node.
    target_column (str): The target column name.
    """
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    learning_rate: int
    random_state: int
    min_samples_split: int
    subsample: int
    min_samples_leaf: int
    target_column: str


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    """
    Class for managing configuration files and retrieving model trainer configuration.

    Attributes:
    config_filepath (str): The file path of the main configuration file.
    params_filepath (str): The file path of the parameters file.
    schema_filepath (str): The file path of the schema file.
    """

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        """
        Initialize ConfigurationManager with file paths and read configuration files.

        Args:
        config_filepath (str): The file path of the main configuration file.
        params_filepath (str): The file path of the parameters file.
        schema_filepath (str): The file path of the schema file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        Retrieve the model trainer configuration from the main configuration.

        Returns:
        ModelTrainerConfig: The model trainer configuration object.
        """
        config = self.config.model_trainer
        params = self.params.GBMClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,

            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            learning_rate=params.learning_rate,
            random_state=params.random_state,
            subsample=params.subsample,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,

            target_column=schema.name
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from mlProject import logger
from sklearn.ensemble import  GradientBoostingClassifier

import joblib
from mlProject.utils.common import read_yaml, create_directories, evaluate_clf

#from mlProject.utils.common import evaluate_clf

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from imblearn.pipeline import  make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold


In [9]:
class ModelTrainer:
    """
    Class for training machine learning models.

    Attributes:
    config (ModelTrainerConfig): The configuration for model training.
    """

    def __init__(self, config: ModelTrainerConfig):
        """
        Initialize ModelTrainer with a configuration object.

        Args:
        config (ModelTrainerConfig): The configuration for model training.
        """
        self.config = config

    def initiate_model_trainer(self):
        """
        Initiate model training using the specified configuration.
        """
        logger.info("Initiating model training")

        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.iloc[:, :-1]
        X_test = test_data.iloc[:, :-1]
        y_train = train_data.iloc[:, -1]
        y_test = test_data.iloc[:, -1]

        GBM = GradientBoostingClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            learning_rate=self.config.learning_rate,
            random_state=self.config.random_state,
            subsample=self.config.subsample,
            min_samples_split=self.config.min_samples_split,
            min_samples_leaf=self.config.min_samples_leaf
        )

        GBM.fit(X_train, y_train)

        joblib.dump(GBM, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    # Initialize ConfigurationManager and retrieve model trainer configuration
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()

    # Initialize ModelTrainer with the retrieved configuration
    model_trainer = ModelTrainer(config=model_trainer_config)

    # Initiate model training
    model_trainer.initiate_model_trainer()

except Exception as e:
    # Raise the caught exception
    raise e


[2024-02-28 11:24:04,105: INFO: common: YAML file loaded successfully from: config\config.yaml]
[2024-02-28 11:24:04,108: INFO: common: YAML file loaded successfully from: params.yaml]
[2024-02-28 11:24:04,111: INFO: common: YAML file loaded successfully from: schema.yaml]
[2024-02-28 11:24:04,112: INFO: common: Created directory at: artifacts]
[2024-02-28 11:24:04,113: INFO: common: Created directory at: artifacts/model_trainer]
[2024-02-28 11:24:04,114: INFO: 2247585756: Initiating model training]
